# Диалоговый бот

## Проверяем карту

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Установка окружения

In [2]:
# Для модели GPT3
pip install transformers==4.6.1

     |████████████████████████████████| 2.3MB 29.8MB/s 
     |████████████████████████████████| 901kB 29.3MB/s 
     |████████████████████████████████| 3.3MB 34.5MB/s 


In [3]:
# Установка библиотек для воспроизведения голоса
!pip install -q torchaudio omegaconf

     |████████████████████████████████| 1.9MB 24.3MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 645kB 41.0MB/s 
     |████████████████████████████████| 112kB 39.4MB/s 


In [21]:
# Распознавание аудио
!pip install SpeechRecognition==3.8.1
!pip install pydub==0.25.1

## Установка окружения для GPU (можно не ставить)

In [ ]:
%%writefile setup.sh
export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
cd apex
git reset --hard a651

pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

Writing setup.sh


In [ ]:
# Нужно устанавливать, только если запускать на GPU, но это не обязательно. Ставится долго
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 8048, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 8048 (delta 65), reused 93 (delta 41), pack-reused 7913
Receiving objects: 100% (8048/8048), 14.10 MiB | 28.10 MiB/s, done.
Resolving deltas: 100% (5466/5466), done.
HEAD is now at a651e2c sync-free Distributed LAMB + parameter reordering (#1055)
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-9q2ur2pj
Created temporary directory: /tmp/pip-req-tracker-x2pk0q2l
Created requirements tracker '/tmp/pip-req-tracker-x2pk0q2l'
Created temporary directory: /tmp/pip-install-lftw_mww
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-dbuo8vmi
  Added file

## Инициализируем модель генерации голоса

In [14]:
import torch
from pprint import pprint
from omegaconf import OmegaConf
from IPython.display import Audio

import warnings

# torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
#                                'latest_silero_models.yml',
#                                progress=False)
# models = OmegaConf.load('latest_silero_models.yml')

In [16]:
# голоса - ['aidar_8khz', 'aidar_16khz', 'baya_8khz', 'baya_16khz', 'irina_8khz', 'irina_16khz', 'kseniya_8khz', 'kseniya_16khz', 'natasha_8khz', 'natasha_16khz', 'ruslan_8khz', 'ruslan_16khz']
language = 'ru'
speaker = 'aidar_16khz'
device = "cuda" if torch.cuda.is_available() else "cpu" #torch.device('cpu')
voice_model, symbols, sample_rate, example_text, apply_tts = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                      model='silero_tts',
                                                                      language=language,
                                                                      speaker=speaker)
voice_model = voice_model.to(device)

# Убираем  предупреждение о длине текста
warnings.filterwarnings("ignore")

Using cache found in /root/.cache/torch/hub/snakers4_silero-models_master


## Иницилазируем библиотеки записи голоса

In [30]:
# Запись с микрофона
import time
import speech_recognition as sr
from IPython.display import Audio
from IPython import display as IPdisplay
import base64
import io
import tempfile
import librosa

from pydub import AudioSegment

from google.colab import files
from google.colab import output
r = sr.Recognizer()

DEFAULT_SAMPLE_RATE = 16000

In [34]:
def record_audio(seconds=3,
                 sample_rate=DEFAULT_SAMPLE_RATE,
                 normalize_db=0.1):
    """Record audio from the browser in colab using javascript.
    Based on: https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be
    Args:
      seconds: Number of seconds to record.
      sample_rate: Resample recorded audio to this sample rate.
      normalize_db: Normalize the audio to this many decibels. Set to None to skip
        normalization step.
    Returns:
      An array of the recorded audio at sample_rate.
    """
    # from IPython import display
    # Use Javascript to record audio.
    record_js_code = """
      const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
      const b2text = blob => new Promise(resolve => {
        const reader = new FileReader()
        reader.onloadend = e => resolve(e.srcElement.result)
        reader.readAsDataURL(blob)
      })
      var record = time => new Promise(async resolve => {
        stream = await navigator.mediaDevices.getUserMedia({ audio: true })
        recorder = new MediaRecorder(stream)
        chunks = []
        recorder.ondataavailable = e => chunks.push(e.data)
        recorder.start()
        await sleep(time)
        recorder.onstop = async ()=>{
          blob = new Blob(chunks)
          text = await b2text(blob)
          resolve(text)
        }
        recorder.stop()
      })
      """
    print(f'Говорите в течение {seconds} секунд...')
    IPdisplay.display(IPdisplay.Javascript(record_js_code))
    audio_string = output.eval_js('record(%d)' % (seconds * 1000.0))
    print('Записано!')
    audio_bytes = base64.b64decode(audio_string.split(',')[1])
    return audio_bytes_to_np(audio_bytes,
                             sample_rate=sample_rate,
                             normalize_db=normalize_db)


def audio_bytes_to_np(wav_data,
                      sample_rate=DEFAULT_SAMPLE_RATE,
                      normalize_db=0.1):
    """Convert audio file data (in bytes) into a numpy array.
    Saves to a tempfile and loads with librosa.
    Args:
      wav_data: A byte stream of audio data.
      sample_rate: Resample recorded audio to this sample rate.
      normalize_db: Normalize the audio to this many decibels. Set to None to skip
        normalization step.
    Returns:
      An array of the recorded audio at sample_rate.
    """
    # Parse and normalize the audio.
    audio = AudioSegment.from_file(io.BytesIO(wav_data))
    audio.remove_dc_offset()
    if normalize_db is not None:
        audio.normalize(headroom=normalize_db)
    # Save to tempfile and load with librosa.
    with tempfile.NamedTemporaryFile(suffix='.wav') as temp_wav_file:
        fname = temp_wav_file.name
    return audio.export(fname, format='wav')

## Подключаем диск для загрузки модели

In [4]:
MODEL_DOWNLOAD_PATH='/content/gdrive/MyDrive/chats_emotions_and_voises/psy_chatbot/models'

In [9]:
# Обеспечиваем подгрузку данных и их хранение в каталоге ноутубка MyDrive/chats_emotions_and_voises/chat04_depression-therapist-chatbot
import os
from google.colab import drive
drive.mount('/content/gdrive')
if not os.path.exists(MODEL_DOWNLOAD_PATH):
    raise ValueError('Нет папки для хранения данных', MODEL_DOWNLOAD_PATH)
%ls $MODEL_DOWNLOAD_PATH

Mounted at /content/gdrive
checkpoin60.zip   checkpoint70.zip      saved_model70000.zip
checkpoint47.zip  saved_model47000.zip


## Скачиваем модель GPT3 чатбота

In [10]:
MODEL_PATH = '/content/saved_model'
LAST_MODEL_ZIP = 'saved_model70000.zip' # Нужно задать архив который проверяется

In [11]:
# Распаковываем архив с моделью, архив содержит папку `saved_model`
!unzip $MODEL_DOWNLOAD_PATH/$LAST_MODEL_ZIP -d '/content' 
%ls

Archive:  /content/gdrive/MyDrive/chats_emotions_and_voises/psy_chatbot/models/saved_model70000.zip
   creating: /content/saved_model/
  inflating: /content/saved_model/config.json  
  inflating: /content/saved_model/tokenizer_config.json  
  inflating: /content/saved_model/tokenizer.json  
  inflating: /content/saved_model/special_tokens_map.json  
  inflating: /content/saved_model/merges.txt  
  inflating: /content/saved_model/pytorch_model.bin  
  inflating: /content/saved_model/training_args.bin  
  inflating: /content/saved_model/vocab.json  
 extracting: /content/saved_model/added_tokens.json  
gdrive/  sample_data/  saved_model/


In [13]:
# Проверяем, что все файлы присутствуют
if not os.path.isdir(MODEL_PATH):
  raise Exception(f"Path '{MODEL_PATH}' not found!")

for fn in ['config.json', 'pytorch_model.bin', 'vocab.json', 'tokenizer_config.json', 'special_tokens_map.json', 'merges.txt']:
  if not os.path.isfile(os.path.join(MODEL_PATH, fn)):
    raise Exception(f'On path "{MODEL_PATH}" file "{fn}" not found!')

## Инициализируем библиотеки GPT3

In [24]:
import os
import random
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)

## Устанавливаем параметры модели чатбота


In [25]:
#@title Параметры модели

# Поле 'Max length tokens generate' должно быть больше 8, но меньше 256
max_length =  100#@param {type:"integer"} # 256
# def 3 Поле 'No repeat ngram size' должно быть больше или равно 1, но меньше или равно 10.
no_repeat_ngram_size = 2 #@param {type:"integer"}
# def 100 Поле 'Top K' должно быть больше или равно 1, но меньше или равно 500. Текущее значение:  100
top_k = 100 #@param {type:"integer"}
# def 0.9 Поле 'Top P' должно быть больше или равно 0.01, но меньше или равно 1.0.
top_p = 0.95 #@param {type:"number"} #0.9
# def 0.6 Поле 'Temperature' должно быть больше или равно 0.01, но меньше или равно 1.0. 
temperature = 0.6 #@param {type:"number"}
# def 5. Поле 'Num responses return' должно быть больше или равно 1, но меньше или равно 10. 
num_return = 5 #@param {type:"integer"}
# def True
do_sample = True  #@param {type:"boolean"}
# def True
is_always_use_length = True  #@param {type:"boolean"}
# Поле 'Length generate' должно принимать одно из следующих значений: [0, 1, 2, 3].
length_gen = 1  #@param {type:"integer"}
# def False. debug
log_debug = True  #@param {type:"boolean"}

params = { 
        "max_length": max_length,
        "no_repeat_ngram_size": no_repeat_ngram_size, 
        "top_k": top_k,                              
        "top_p": top_p,                              
        "temperature": temperature,                   
        "num_return_sequences": num_return,           
        "do_sample": do_sample,
        "device": "cuda" if torch.cuda.is_available() else "cpu", #'cuda:0', # 'cpu',
        "is_always_use_length": is_always_use_length,
        "length_generate": length_gen, 
        "log_debug": log_debug
    }
print(params)

{'max_length': 100, 'no_repeat_ngram_size': 2, 'top_k': 100, 'top_p': 0.95, 'temperature': 0.6, 'num_return_sequences': 5, 'do_sample': True, 'device': 'cpu', 'is_always_use_length': True, 'length_generate': 1, 'log_debug': True}


## Функции чатбота

In [26]:
# Функция определения длины диалога
def get_length_param(text: str, model_tokenizer) -> str:
    tokens_count = len(model_tokenizer.encode(text))
    if tokens_count <= 15:
        len_param = '1'
    elif tokens_count <= 50:
        len_param = '2'
    elif tokens_count <= 256:
        len_param = '3'
    else:
        len_param = '-'
    return len_param

In [27]:
# Генерация ответа
def generate(gpt3model, full_inputs, params, dialog_tokenizer): # inputs = [{'speaker': 0, 'text': 'Привет, как день прошел?'},{'speaker': 1, 'text': 'Хорошо, а у тебя как?'}]
  if len(full_inputs) > 5: # Ограничиваем кол-во истории в диалоге на 2 предыдущих + вопрос
    inputs = full_inputs[-5:]
  else:
    inputs = full_inputs
  inputs_text = ''
  for input_ in inputs:
    if params['is_always_use_length']:
        length_param = get_length_param(input_['text'], dialog_tokenizer)
    else:
        length_param = '-'
    inputs_text += f"|{input_['speaker']}|{length_param}|{input_['text']}"
  inputs_text += f"|1|{params['length_generate']}|"

  if params['log_debug']:
    print(f"        [debug] Затравка вопроса для модели: {inputs_text}")
  try:
    inputs_token_ids = dialog_tokenizer.encode(inputs_text, return_tensors='pt', add_special_tokens=False) # added add_special_tokens=False, 
    inputs_token_ids = dialog_tokenizer.encode(full_inputs[-1]['text'], return_tensors='pt', add_special_tokens=False) # added add_special_tokens=False, 
    # print('$$$$', len(inputs_token_ids),'!', inputs_token_ids, '$$$')
    inputs_token_ids = inputs_token_ids.to(params['device']) # Ошибка должны быть генерированы на одном устройстве
  except Exception as e:
    print('Ошибка', e)
  try:
    # ToDo make this asynchronous
    outputs_token_ids = gpt3model.generate(
        input_ids=inputs_token_ids,
        # inputs_token_ids,
        max_length=params['max_length'] + len(inputs_token_ids[0]),
        no_repeat_ngram_size=params['no_repeat_ngram_size'],
        do_sample=params['do_sample'],
        top_k=params['top_k'],
        top_p=params['top_p'],
        temperature=params['temperature'],
        num_return_sequences=params['num_return_sequences'],
        device=params['device'],
        mask_token_id=dialog_tokenizer.mask_token_id,
        eos_token_id=dialog_tokenizer.eos_token_id,
        unk_token_id=dialog_tokenizer.unk_token_id,
        pad_token_id=dialog_tokenizer.pad_token_id,
    )
  except Exception as e:
    print(f"===> Error generate: ", e) # {str(e)}
    return {'inputs': '', 'outputs': '', 'status': False, 'msg': f"{str(e)}"}

  outputs = [dialog_tokenizer.decode(x, skip_special_tokens=True) for x in outputs_token_ids]
  outputs = [x.split('|')[-1] for x in outputs]

  return {'inputs': inputs, 'outputs': outputs, 'status': True, 'msg': ''}

## Загружаем модель

In [28]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
#MODEL_PATH = 'sberbank-ai/rugpt3small_based_on_gpt2' # Если проверяем работоспособность на базовой GPT, а не на обученной модели
dialog_tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
dialog_model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)




In [36]:
# вводить текстом
print('Скажите "стоп" для остановки. Говорите фразу длиной до 3х секунд')
if params['log_debug']:
  print(f"\n===> debug Params generate: {params}\n\n")
inputs = []
while True:
  # Текстовый ввод
  # user_input = input("USER:")

  # Запись с микрофона браузера
  with sr.WavFile(record_audio(seconds=3)) as source:
      input_audio = r.record(source)
  try:
    user_input = r.recognize_google(input_audio, language="ru-RU") # Ошибка если запись пустая
  except Exception as err:
    print('Не распознан голос. Возможно вы ничего не сказали? Попробуйте еще раз.')
    time.sleep(2)
    continue

  print('USER:', user_input)
  
  # if user_input == "quit":
  if user_input == "стоп":
    print ("Останавливаем диалог!")
    break
  
  inputs.append({'speaker': 0, 'text': user_input})
  response_data = generate(dialog_model, inputs, params, dialog_tokenizer) 
  if not response_data['status']:
    print("BOT (ошибка):", response_data['msg'])
  else:
    bot_resposnse = response_data['outputs'][0]
    inputs.append({'speaker': 1, 'text': bot_resposnse})
    print("\n>>>BOT:", bot_resposnse)
    variants_responses = response_data['outputs']
    if params['log_debug']:
      print("        [debug] Другие варианты ответа (можно выбирать отдельным алгоритмом наиболее подходящий) ===> :", variants_responses[1:])
  text_bot = bot_resposnse
  audio = apply_tts(texts= text_bot,
                    model=voice_model,
                    sample_rate=sample_rate,
                    symbols=symbols,
                    device=device)
  sample_length = librosa.get_duration(audio[0])
  display(Audio(audio[0], rate=sample_rate, autoplay=True))
  time.sleep(sample_length + 5)

Введите "стоп" для остановки. Говорите фразу длиной до 3х секунд

===> debug Params generate: {'max_length': 100, 'no_repeat_ngram_size': 2, 'top_k': 100, 'top_p': 0.95, 'temperature': 0.6, 'num_return_sequences': 5, 'do_sample': True, 'device': 'cpu', 'is_always_use_length': True, 'length_generate': 1, 'log_debug': True}


Говорите в течение 3 секунд...


<IPython.core.display.Javascript object>

Записано!


Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


Не распознан голос. Возможно вы ничего не сказали? Попробуйте еще раз
USER: Привет! Как дела?
        [debug] Затравка вопроса для модели: |0|1|Привет! Как дела?|1|1|

>>>BOT: Привет. Нормально. Все хорошо. Но я боюсь что у меня нет денег на то, чтобы снять квартиру, потому что не смогу оплачивать ипотеку. А в остальном я просто живу и все. И учусь, работаю, езжу на работу. Я не хочу рушить семью. Мне не нравится то что я живу с родителями, мне не хочется видеть их. Меня пугает то как они относятся к моей работе. Они не будут помогать, не поддержат
        [debug] Другие варианты ответа (можно выбирать отдельным алгоритмом наиболее подходящий) ===> : ['- Как считаете, в', 'Не знаю, как у вас, у нас в РФ все одинаково.', 'Такое впечатление,что он думает, будто вы с ребенком не общаемся,а он вам дает возможность общаться.\n\n', ' Брусника34 писал(а): А в чем проблема']


...

<IPython.core.display.Javascript object>

Записано!


Setting `pad_token_id` to `eos_token_id`:50257 for open-end generation.


Не распознан голос. Возможно вы ничего не сказали? Попробуйте еще раз
USER: Привет! Как дела?
        [debug] Затравка вопроса для модели: |0|1|Привет! Как дела?|1|3|Привет. Нормально. Все хорошо. Но я боюсь что у меня нет денег на то, чтобы снять квартиру, потому что не смогу оплачивать ипотеку. А в остальном я просто живу и все. И учусь, работаю, езжу на работу. Я не хочу рушить семью. Мне не нравится то что я живу с родителями, мне не хочется видеть их. Меня пугает то как они относятся к моей работе. Они не будут помогать, не поддержат|0|1|Привет! Как дела?|1|1|


KeyboardInterrupt: ignored